<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Categorical_Feature_Encoding_Challenge_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
# !kaggle competitions list

In [ ]:
!kaggle competitions download -c cat-in-the-dat

cat-in-the-dat.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

 cat-in-the-dat.zip   kaggle.json   sample_submission.csv   train.csv
'kaggle (1).json'     sample_data   test.csv


In [ ]:
!unzip cat-in-the-dat.zip

Archive:  cat-in-the-dat.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

## **1.1. 데이터 준비**

데이터를 준비하겠습니다. 'id' 피처의 고윳값 개수가 레코드 개수와 일치하므로 'id' 피처를 인덱스로 삼겠습니다.

In [ ]:
import numpy as np
import pandas as pd

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다. 'id' 피처는 인덱스로 지정합니다.
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
submission = pd.read_csv('sample_submission.csv', index_col='id')

훈련 세트와 테스트 세트를 위아래로 쌓고 데이터 정보를 확인하겠습니다.

In [ ]:
all_data = pd.concat([train, test], ignore_index=False)
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,F,N,Green,Square,Lion,Canada,Theremin,6c90f81cd,a406a5f12,d02a6b0ba,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3,NaN
499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,Piano,2d61990e2,520806ce2,d2d8eabdb,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2,NaN
499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,Oboe,488406659,28645754b,22831fffe,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1,NaN


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500000 entries, 0 to 499999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   bin_0   500000 non-null  int64  
 1   bin_1   500000 non-null  int64  
 2   bin_2   500000 non-null  int64  
 3   bin_3   500000 non-null  object 
 4   bin_4   500000 non-null  object 
 5   nom_0   500000 non-null  object 
 6   nom_1   500000 non-null  object 
 7   nom_2   500000 non-null  object 
 8   nom_3   500000 non-null  object 
 9   nom_4   500000 non-null  object 
 10  nom_5   500000 non-null  object 
 11  nom_6   500000 non-null  object 
 12  nom_7   500000 non-null  object 
 13  nom_8   500000 non-null  object 
 14  nom_9   500000 non-null  object 
 15  ord_0   500000 non-null  int64  
 16  ord_1   500000 non-null  object 
 17  ord_2   500000 non-null  object 
 18  ord_3   500000 non-null  object 
 19  ord_4   500000 non-null  object 
 20  ord_5   500000 non-null  object 
 21  day     50

테스트 세트에는 'target' 피처가 없습니다. 'target' 피처를 제거하겠습니다.

In [ ]:
all_data = all_data.drop('target', axis=1)

`OneHotEncoder` 클래스를 사용하여 원-핫 인코딩을 수행하겠습니다. 물론 (이 데이터에 한정하여) 판다스의 `get_dummies()` 메서드를 사용해도 무방합니다.

참고로 `get_dummies()` 메서드는 결정적인 단점이 존재합니다. 학습 세트의 피처를 학습하는 과정 없이 그대로 원-핫 인코딩을 수행하기 때문에 학습 세트에만 존재하고 테스트 세트에 없는 값은 원-핫인코딩이 적용된 피처로 변환하지 않습니다. 게다가 `OneHotEncoder` 클래스보다 더 많은 메모리가 필요합니다. 따라서 원-핫 인코딩을 적용하는 경우 습관적으로 `OneHotEncoder` 클래스를 사용하는 편이 좋습니다. [링크](https://psystat.tistory.com/136)에서 상세한 설명을 확인할 수 있습니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
# 데이터를 하나로 합쳤으므로 `fit_transform()` 메서드를 사용합니다.
all_data_encoded = encoder.fit_transform(all_data)



---

업데이트 중

---

